# CMSC 35300 Final Project
## Jacob Jameson & Hugo Salas

### Creating an Autoencoder for Gene Expression Data

**In this notebook, we build an autoencoder neural network that takes gene expression data from protein-coding genes only, encodes them through multiple layers of decreasing dimensions, and then decodes them back the original dimensionality as what was inputted.**

In [5]:
import requests
import json
import re
import gzip
import shutil
import pandas as pd
import os
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.manifold import TSNE

import seaborn as sns
%matplotlib inline 

from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler, Normalizer

**In the code below, I import the RNA sequence samples and create a dataframe where each row is one sample and the columns are the genes in that sample. I used 137 female samples to construct my dataset. I first downloaded the data from the portal, so the code below loops through the data already downloaded to open and merge it together.**

In [7]:
directory = "/Users/jacob/Downloads/ML for Medicine/Assignment 2/gdc_download_20211025_201031.006887"
folder_names = []
for filename in os.listdir(directory):
    if not filename.endswith(".DS_Store") and not filename.endswith(".txt"):
        folder_names.append(os.path.join(directory, filename))

file_names = []
for folder in folder_names:
    for filename in os.listdir(folder):
        if filename.endswith(".gz"):
            file_names.append(os.path.join(folder, filename))

In [40]:
for n, file in enumerate(file_names):
    with gzip.open(file, 'rb') as f_in:
        with open(file + '.txt', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            
    data = pd.read_csv(file + '.txt', sep='\t', 
                       header=None).rename(columns={0: "Gene", 1: "Sample"})
    data.drop(data.tail(5).index,inplace=True)
    data = data.rename(columns={"Sample": "Sample " + str(n + 1)})
    if n > 0:
        grouped_df = data.merge(grouped_df, how='left', on='Gene')
    else:
        grouped_df = data
        
grouped_df = grouped_df.set_index('Gene')
final_data = grouped_df.T

In [42]:
final_data

Gene,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,...,ENSGR0000263980.4,ENSGR0000264510.4,ENSGR0000264819.4,ENSGR0000265658.4,ENSGR0000270726.4,ENSGR0000275287.3,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1
Sample 137,1603,1,2036,1609,443,938,21034,6496,4427,2711,...,0,0,0,0,0,0,0,0,0,0
Sample 136,3901,0,4442,1489,1133,212,2426,4387,2457,2646,...,0,0,0,0,0,0,0,0,0,0
Sample 135,393,0,2140,1232,321,153,939,2923,1681,1630,...,0,0,0,0,0,0,0,0,0,0
Sample 134,957,1,3664,1231,735,511,8050,7202,6800,2342,...,0,0,0,0,0,0,0,0,0,0
Sample 133,725,2,1172,654,245,399,2492,2023,1110,656,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sample 5,790,139,2331,654,391,350,4699,2361,1760,1375,...,0,0,0,0,0,0,0,0,0,0
Sample 4,1827,4,3786,1192,760,973,6990,5020,4228,2428,...,0,0,0,0,0,0,0,0,0,0
Sample 3,9582,0,4265,923,485,722,2209,4603,2698,1860,...,0,0,0,0,0,0,0,0,0,0
Sample 2,631,2,2857,716,625,787,6451,4200,2279,2711,...,0,0,0,0,0,0,0,0,0,0


## Pre-Processing

**Only keep genes that are protein encoding genes.**

In [66]:
nt_coding = pd.read_csv('MLiB-Lab3-PartA/nt.coding.csv')
nt_coding.drop('Type', axis=1, inplace=True)

final_data_coding = final_data[list(nt_coding.columns)]

final_data_coding

Gene,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,...,ENSG00000269699.4,ENSG00000269711.1,ENSG00000269741.4,ENSG00000269749.1,ENSG00000269755.1,ENSG00000269846.1,ENSG00000269855.2,ENSG00000269858.4,ENSG00000269881.1,ENSG00000269883.1
Sample 137,1603,1,2036,1609,443,938,21034,6496,4427,2711,...,11,0,0,1,8,7,5,1128,36,1
Sample 136,3901,0,4442,1489,1133,212,2426,4387,2457,2646,...,0,0,0,0,1,0,1,1377,6,0
Sample 135,393,0,2140,1232,321,153,939,2923,1681,1630,...,0,0,0,0,2,0,17,738,34,1
Sample 134,957,1,3664,1231,735,511,8050,7202,6800,2342,...,0,0,10,0,1,10,2,691,6,0
Sample 133,725,2,1172,654,245,399,2492,2023,1110,656,...,9,0,0,0,3,5,2,566,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sample 5,790,139,2331,654,391,350,4699,2361,1760,1375,...,1,0,0,0,1,1,0,857,16,0
Sample 4,1827,4,3786,1192,760,973,6990,5020,4228,2428,...,11,0,5,0,7,7,31,536,6,7
Sample 3,9582,0,4265,923,485,722,2209,4603,2698,1860,...,2,0,0,0,2,1,0,1371,5,0
Sample 2,631,2,2857,716,625,787,6451,4200,2279,2711,...,2,0,1,1,0,12,7,567,16,1


**Normalize the data**

## Autoencoder

In [67]:
X_train, X_test = train_test_split(final_data_coding, 
                                   test_size=0.3, 
                                   random_state=1)

scaler = Normalizer()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [68]:
input_layer = Input(shape=(X_train.shape[1],))
encoded = Dense(256, activation='relu')(input_layer)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(50, activation='relu')(encoded)

decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(X_train.shape[1], activation='sigmoid')(decoded)

In [69]:
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [70]:
autoencoder.fit(X_train, X_train,
                epochs=20,
                batch_size=20)

Epoch 1/20
5/5 [==============================] - 1s 50ms/step - loss: 0.6923
Epoch 2/20
5/5 [==============================] - 0s 50ms/step - loss: 0.6884
Epoch 3/20
5/5 [==============================] - 0s 53ms/step - loss: 0.6620
Epoch 4/20
5/5 [==============================] - 0s 75ms/step - loss: 0.4848
Epoch 5/20
5/5 [==============================] - 0s 50ms/step - loss: 0.1101
Epoch 6/20
5/5 [==============================] - 0s 84ms/step - loss: 0.0129
Epoch 7/20
5/5 [==============================] - 0s 74ms/step - loss: 0.0207
Epoch 8/20
5/5 [==============================] - 0s 51ms/step - loss: 0.0268
Epoch 9/20
5/5 [==============================] - 0s 49ms/step - loss: 0.0277
Epoch 10/20
5/5 [==============================] - 0s 56ms/step - loss: 0.0254
Epoch 11/20
5/5 [==============================] - 0s 51ms/step - loss: 0.0219
Epoch 12/20
5/5 [==============================] - 0s 51ms/step - loss: 0.0186
Epoch 13/20
5/5 [==============================] - 0s 47ms/st

In [71]:
autoencoder.evaluate(X_test, X_test, verbose=2)

2/2 - 0s - loss: 0.0085 - 151ms/epoch - 76ms/step


0.008524371311068535

**Here we explore different bottleneck layers to see how compression effects our loss**

In [72]:
losses = dict()

for size in [10, 20, 30, 40, 60, 80]:
    input_layer = Input(shape=(X_train.shape[1],))
    encoded = Dense(256, activation='relu')(input_layer)
    encoded = Dense(128, activation='relu')(encoded)
    encoded = Dense(size, activation='relu')(encoded)

    decoded = Dense(128, activation='relu')(encoded)
    decoded = Dense(256, activation='relu')(decoded)
    decoded = Dense(X_train.shape[1], activation='sigmoid')(decoded)
    
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    autoencoder.fit(X_train, X_train, epochs=20, batch_size=20)
    
    losses[size] = autoencoder.evaluate(X_test, X_test, verbose=2)

Epoch 1/20
5/5 [==============================] - 1s 46ms/step - loss: 0.6923
Epoch 2/20
5/5 [==============================] - 0s 43ms/step - loss: 0.6894
Epoch 3/20
5/5 [==============================] - 0s 45ms/step - loss: 0.6802
Epoch 4/20
5/5 [==============================] - 0s 41ms/step - loss: 0.6207
Epoch 5/20
5/5 [==============================] - 0s 46ms/step - loss: 0.3645
Epoch 6/20
5/5 [==============================] - 0s 42ms/step - loss: 0.0571
Epoch 7/20
5/5 [==============================] - 0s 80ms/step - loss: 0.0128
Epoch 8/20
5/5 [==============================] - 0s 68ms/step - loss: 0.0190
Epoch 9/20
5/5 [==============================] - 0s 59ms/step - loss: 0.0231
Epoch 10/20
5/5 [==============================] - 0s 48ms/step - loss: 0.0238
Epoch 11/20
5/5 [==============================] - 0s 43ms/step - loss: 0.0223
Epoch 12/20
5/5 [==============================] - 0s 46ms/step - loss: 0.0201
Epoch 13/20
5/5 [==============================] - 0s 50ms/st

5/5 [==============================] - 0s 42ms/step - loss: 0.0155
Epoch 14/20
5/5 [==============================] - 0s 42ms/step - loss: 0.0133
Epoch 15/20
5/5 [==============================] - 0s 50ms/step - loss: 0.0119
Epoch 16/20
5/5 [==============================] - 0s 57ms/step - loss: 0.0108
Epoch 17/20
5/5 [==============================] - 0s 59ms/step - loss: 0.0101
Epoch 18/20
5/5 [==============================] - 0s 60ms/step - loss: 0.0097
Epoch 19/20
5/5 [==============================] - 0s 57ms/step - loss: 0.0095
Epoch 20/20
5/5 [==============================] - 0s 45ms/step - loss: 0.0093
2/2 - 0s - loss: 0.0085 - 140ms/epoch - 70ms/step
Epoch 1/20
5/5 [==============================] - 1s 70ms/step - loss: 0.6922
Epoch 2/20
5/5 [==============================] - 0s 61ms/step - loss: 0.6880
Epoch 3/20
5/5 [==============================] - 0s 71ms/step - loss: 0.6579
Epoch 4/20
5/5 [==============================] - 0s 71ms/step - loss: 0.4678
Epoch 5/20
5/5 [==

In [73]:
losses

{10: 0.008590234443545341,
 20: 0.008578382432460785,
 30: 0.008572423830628395,
 40: 0.008356384932994843,
 60: 0.008510102517902851,
 80: 0.008471248671412468}